# Day 7 - Lab 1: Advanced Agent Workflows with MCP

**Objective:** Master the Model Context Protocol (MCP) to build reliable, predictable agents for automating new employee onboarding workflows.

**Estimated Time:** 135 minutes

## Introduction

Today you'll learn how MCP transforms unreliable AI agents into production-ready systems by building an onboarding automation agent.

### Why MCP?

Without structure, AI agents make inconsistent decisions. MCP provides:
1. **Resources**: Structured, versioned data sources
2. **Tools**: Validated, idempotent operations
3. **Schema Enforcement**: Type-safe interactions

## Step 1: Environment Setup

In [ ]:
import sys
import os
import json
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Any
from enum import Enum

# Add project root to path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# MCP Import Guard
MCP_AVAILABLE = False
try:
    from mcp.server import Server
    from mcp.server.fastmcp import FastMCP
    MCP_AVAILABLE = True
    print('✅ MCP imports successful')
except ImportError as e:
    print('⚠️ MCP not available. Install with: pip install mcp')
    print('   Lab will continue in demonstration mode.')

from utils import setup_llm_client, get_completion, save_artifact
client, model_name, api_provider = setup_llm_client(model_name='gpt-4o')

## Step 2: Load Onboarding Data

We'll work with real onboarding data including policies, access requirements, and training catalogs.

In [ ]:
# Load onboarding data assets
assets_path = 'assets'

with open(f'{assets_path}/onboarding_docs.json', 'r') as f:
    onboarding_docs = json.load(f)

with open(f'{assets_path}/roles_access_matrix.json', 'r') as f:
    roles_access = json.load(f)

with open(f'{assets_path}/training_catalog.json', 'r') as f:
    training_catalog = json.load(f)

with open(f'{assets_path}/new_hires_sample.json', 'r') as f:
    new_hires = json.load(f)

print(f"Loaded {len(onboarding_docs['policies'])} policies")
print(f"Loaded {len(roles_access['roles'])} role definitions")
print(f"Loaded {len(training_catalog['courses'])} training courses")
print(f"Loaded {len(new_hires['new_hires'])} sample new hires")

## Step 3: Define Onboarding Schemas

Strong typing ensures our agent makes consistent decisions.

In [ ]:
@dataclass
class HireProfile:
    id: str
    name: str
    email: str
    role: str
    start_date: str
    manager: str
    location: str
    
@dataclass
class AccessRequest:
    system: str
    hire_id: str
    role: str
    justification: str
    status: str = 'pending'
    created_at: str = field(default_factory=lambda: datetime.now().isoformat())

@dataclass
class TrainingItem:
    course_id: str
    title: str
    duration: str
    due_date: str
    status: str = 'not_started'

@dataclass 
class OnboardingPlan:
    hire_id: str
    access_requests: List[AccessRequest] = field(default_factory=list)
    training_items: List[TrainingItem] = field(default_factory=list)
    checklist_items: List[Dict] = field(default_factory=list)

# Test schema validation
test_hire = HireProfile(**new_hires['new_hires'][0])
print(f"✅ Schema validation successful for {test_hire.name}")

## Challenge 1 (Foundational): Build MCP Server with Resources

**Task:** Create an MCP server that exposes onboarding data as structured resources.

**TODO:** Complete the resource implementations below.

In [ ]:
if MCP_AVAILABLE:
    # Initialize MCP server
    mcp_server = FastMCP("onboarding-server")
    
    # TODO: Implement the docs resource
    @mcp_server.resource("onboarding/docs")
    async def get_onboarding_docs() -> Dict[str, Any]:
        """Return onboarding documentation and policies."""
        # YOUR CODE HERE
        pass
    
    # TODO: Implement the roles resource  
    @mcp_server.resource("onboarding/roles")
    async def get_roles_matrix() -> Dict[str, Any]:
        """Return role-based access requirements."""
        # YOUR CODE HERE
        pass
    
    # TODO: Implement the training resource
    @mcp_server.resource("onboarding/training") 
    async def get_training_catalog() -> Dict[str, Any]:
        """Return available training courses."""
        # YOUR CODE HERE
        pass
    
    print("MCP Server initialized with resources")
else:
    print("Skipping MCP server setup (MCP not available)")

## Challenge 2 (Intermediate): Implement MCP Tools

**Task:** Add tools for creating access requests and scheduling training.

In [ ]:
# Storage for requests (in production, use a database)
access_requests_db = []
training_schedules_db = []

if MCP_AVAILABLE:
    # TODO: Implement the create_access_request tool
    @mcp_server.tool()
    async def create_access_request(system: str, hire_id: str, role: str) -> Dict[str, Any]:
        """Create an access request for a new hire."""
        # YOUR CODE HERE
        # 1. Check if request already exists (idempotency)
        # 2. Create AccessRequest object
        # 3. Add to access_requests_db
        # 4. Return confirmation
        pass
    
    # TODO: Implement the schedule_training tool
    @mcp_server.tool()
    async def schedule_training(hire_id: str, course_ids: List[str]) -> Dict[str, Any]:
        """Schedule training courses for a new hire."""
        # YOUR CODE HERE
        # 1. Validate course IDs exist
        # 2. Calculate due dates
        # 3. Create TrainingItem objects
        # 4. Add to training_schedules_db
        pass
    
    print("MCP Tools registered")
else:
    print("Skipping MCP tools (MCP not available)")

## Challenge 3 (Advanced): Reliability Demonstration

**Task:** Compare MCP-based vs unstructured approaches to show deterministic behavior.

In [ ]:
import random

def check_soc2_requirement_unstructured(role: str, context: str) -> bool:
    """Unreliable: Makes decision based on unstructured context."""
    # Simulate inconsistent interpretation
    if 'maybe' in context.lower() or 'might' in context.lower():
        return random.choice([True, False])
    return 'soc2' in context.lower() or role == 'engineer'

def check_soc2_requirement_mcp(role: str) -> bool:
    """Reliable: Uses structured MCP resource."""
    # Always check authoritative source
    for policy in onboarding_docs['policies']:
        if policy['id'] == 'POL-003':  # SOC2 policy
            return role in policy['required_for_roles']
    return False

# Test consistency
test_role = 'engineer'
ambiguous_context = 'The new engineer might need SOC2 training, maybe check with compliance'

# Run 5 times each
unstructured_results = []
mcp_results = []

for _ in range(5):
    unstructured_results.append(check_soc2_requirement_unstructured(test_role, ambiguous_context))
    mcp_results.append(check_soc2_requirement_mcp(test_role))

print(f"Unstructured results: {unstructured_results} - Consistent: {len(set(unstructured_results)) == 1}")
print(f"MCP results: {mcp_results} - Consistent: {len(set(mcp_results)) == 1}")

# TODO: Add assertion to verify MCP is always consistent
# assert len(set(mcp_results)) == 1, "MCP should always be consistent"

## Self-Assessment Checklist

Run this cell to verify your implementation:

In [ ]:
def run_self_checks():
    checks_passed = []
    
    # Check 1: Data loaded
    try:
        assert len(onboarding_docs['policies']) > 0
        checks_passed.append('✅ Onboarding data loaded')
    except:
        checks_passed.append('❌ Onboarding data not loaded')
    
    # Check 2: Schemas defined
    try:
        test = HireProfile(id='test', name='Test', email='test@example.com',
                          role='engineer', start_date='2024-01-01',
                          manager='Manager', location='Remote')
        checks_passed.append('✅ Schemas properly defined')
    except:
        checks_passed.append('❌ Schema definitions incomplete')
    
    # Check 3: MCP available
    if MCP_AVAILABLE:
        checks_passed.append('✅ MCP library available')
    else:
        checks_passed.append('⚠️ MCP library not installed')
    
    # Check 4: Consistency test
    test_results = [check_soc2_requirement_mcp('engineer') for _ in range(10)]
    if len(set(test_results)) == 1:
        checks_passed.append('✅ MCP consistency verified')
    else:
        checks_passed.append('❌ MCP consistency check failed')
    
    print('\n'.join(checks_passed))
    return all('✅' in check for check in checks_passed if '⚠️' not in check)

all_passed = run_self_checks()
print(f"\nAll required checks passed: {all_passed}")

## Conclusion

Congratulations! You've learned how MCP transforms agent reliability through:

1. **Structured Resources**: Consistent data access
2. **Validated Tools**: Type-safe, idempotent operations  
3. **Schema Enforcement**: Predictable data structures

### Key Takeaways:
- MCP eliminates ambiguity in agent context
- Structured resources ensure deterministic decisions
- Tools with validation prevent errors

### Next Steps:
- Proceed to Lab 2 to learn about A2A (Agent-to-Agent) communication
- Explore combining MCP with A2A for multi-agent systems